In [3]:
import pandas as pd
import re
import numpy as np

In [10]:
def extract_markers_from_file(marker_names, new_marker_names, file_path, new_file_path, anchor_marker="", anchor_vector=0):

    # --- Header einlesen ---
    with open(file_path, "r") as f:
        header_lines = [next(f) for _ in range(5)]

    # --- Marker-Header und Achsen einlesen ---
    marker_header = pd.read_csv(file_path, sep="\t", skiprows=3, nrows=1, header=None)
    coord_header  = pd.read_csv(file_path, sep="\t", skiprows=4, nrows=1, header=None)

    columns = []
    current_marker = None

    for m, c in zip(marker_header.values[0], coord_header.values[0]):
        if isinstance(m, str) and m.strip() != "":
            current_marker = m.strip()

        if current_marker is None:
            columns.append(c)
        else:
            axis = re.sub(r"\d+", "", str(c))
            columns.append(f"{current_marker}_{axis}")

    # Fix für erste zwei Spalten
    columns[0] = "Frame#"
    columns[1] = "Time"

    df = pd.read_csv(file_path, sep="\t", skiprows=5, names=columns)

    # --- Gewünschte Marker auswählen ---
    keep_cols = ["Frame#", "Time"]
    for name in marker_names:
        keep_cols += [f"{name}_X", f"{name}_Y", f"{name}_Z"]

    filtered_df = df[keep_cols]

    origin_d = df.loc[0, [anchor_marker + "_X", anchor_marker + "_Y", anchor_marker + "_Z"]]
    origin_d = np.array(origin_d) - anchor_vector

    corrected_df = filtered_df.head(0).copy()

    for i in range(len(filtered_df)):
        row = filtered_df.iloc[i].copy()

        # correct origin displacement
        for m in range(len(marker_names)):
            marker = marker_names[m]
            v = np.array([row[marker + "_X"], row[marker + "_Y"], row[marker + "_Z"]]) - origin_d
            row[marker + "_X"] = v[0]
            row[marker + "_Y"] = v[1]
            row[marker + "_Z"] = v[2]
        corrected_df.loc[len(corrected_df)] = row
        
    # --- Header anpassen ---

    # 1. NumMarkers
    header_lines[2] = re.sub(r"\d+\s+m", f"{len(marker_names)}\tm", header_lines[2])

    # 2. Marker-Namen-Zeile (Zeile 4)
    marker_line = ["Frame#", "Time"]
    for name in new_marker_names:
        marker_line.append(name)
        marker_line.append("")  # Tab zwischen X/Y/Z
        marker_line.append("")  # Tab zwischen X/Y/Z
    header_lines[3] = "\t".join(marker_line) + "\n"

    # 3. Koordinaten-Zeile (Zeile 5)
    coord_line = []
    counter = 1
    for _ in marker_names:
        coord_line += ["X" + str(counter), "Y" + str(counter), "Z" + str(counter)]
        counter += 1
    header_lines[4] = "\t".join(coord_line) + "\n"

    # --- Neue TRC-Datei schreiben ---
    with open(new_file_path, "w") as f:
        for line in header_lines:
            f.write(line)

    corrected_df["Frame#"] = corrected_df["Frame#"].astype(int)
    corrected_df.to_csv(new_file_path, sep="\t", mode="a", index=False, header=False)

    print(f"Neue TRC gespeichert unter: {new_file_path}")


In [15]:
anchor_vector = np.array([0,0.837,0.17])
anchor_marker = "RShoulder"

extract_markers_from_file(["RShoulder", "RElbow", "RWrist"], ["r_acromion", "r_humerus_epicondyle", "r_radius_styloid"], "max-elbow-flex-wide_P0_1-783_filt_butterworth_LSTM.trc", "pose2sim_dynamic_origin_d.trc", anchor_marker, anchor_vector)

Neue TRC gespeichert unter: pose2sim_dynamic_origin_d.trc
